In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline
import sqlalchemy
import pandas as pd
import numpy as np
import random
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error

import seaborn as sns

from sklearn.preprocessing import StandardScaler

In [2]:
password = 'dimVuidBun1onCub'
login = 'a.al.kosheleva'

def postre_auth(login, password):
    postgresql_config = {
    'postgresql_prefix' : 'postgresql'
    , 'postgresql_host'   : '10.3.26.92' 
    , 'postgresql_user'   : f'{login}'
    , 'postgresql_pass'   : f'{password}'
    , 'postgresql_port'   : '5432'
    , 'postgresql_db'     : 'dwh'    
    }

    postgresql_connection_string  = "{prefix}://{user}:{passw}@{host}:{port}/{db}".format(
    prefix  =   postgresql_config['postgresql_prefix']    , host  =   postgresql_config['postgresql_host']
    , user  =   postgresql_config['postgresql_user']      , passw =   postgresql_config['postgresql_pass']
    , port  =   postgresql_config['postgresql_port']      , db    =   postgresql_config['postgresql_db']
    )
    postgresql_engine = sqlalchemy.create_engine(postgresql_connection_string)
    postgresql_conn   = postgresql_engine.connect()
    return postgresql_engine, postgresql_conn

postrgre_engine, postrge_conn = postre_auth(login, password)

In [3]:
#запрос для выгрузки продаж в категории Чай-кофе-какао за 3 месяца
sql = '''
with skus AS (
select
  distinct substr(mat.material,13,18)  original_id
from dwh_stage.hadoop_bi0_pmaterial mat
inner join dwh_stage.hadoop_bic_rpa_wgh wgh
on mat.rpa_wgh4 = wgh._0rpa_wgh4
where rpa_wgh2 != '99'
and wgh._0rpa_wgh2___t != 'Резервная группа'
and wgh._0rpa_wgh2___t = 'Кофе, какао, чай'
),

orders AS (
select distinct buyer_id, pav_order_id, date(created) as date
from dwh_datamart.ops_metrics
where date(created) between '2023-01-01' AND '2023-04-19'
and flag_cancel = 0 and order_source IN ('iOS', 'Android')
and retail_brand = 'lo'
),

t1 as (
select
  date, zit.original_id,
  sum(zit.count_courier*zit.denominator/zit.numerator) as revenue
from orders z
inner join dwh_stage.utk_sess_zakaz_items zit
on zit.pav_order_id = z.pav_order_id
where zit.original_id IN (select * from skus)
group by 1,2
)

select *, sum(revenue) over(partition by original_id) as total_sku_revenue
from t1
'''

In [86]:
all_cats = pd.read_excel('C:/Users/a.al.kosheleva/analytics/kosheleva_a/all_cats.xlsx')

In [87]:
all_cats.shape

(68311, 5)

In [88]:
all_cats.head()

,Unnamed: 0,date,original_id,revenue,total_sku_revenue
0,0,2023-01-22,729,1.0,1
1,1,2023-01-06,744,4.0,298
2,2,2023-03-04,744,3.0,298
3,3,2023-02-08,744,6.0,298
4,4,2023-02-21,744,1.0,298


In [89]:
all_cats = pd.read_sql(sql, postrge_conn)

NameError: name 'sql' is not defined

In [90]:
all_cats['date'] = pd.to_datetime(all_cats['date'])

In [91]:
#выделяем пилот и предпилот
df_prepilot = all_cats.loc[(all_cats['date'] <= '2023-04-10')].copy()
df_pilot = all_cats.loc[all_cats['date'] > '2023-04-10'].copy()

In [92]:
df_sku_sales = df_prepilot.pivot_table(index = 'original_id', aggfunc = {'revenue':'sum'}).reset_index()

In [93]:
list_sku = df_sku_sales.loc[(df_sku_sales['revenue']>= df_sku_sales['revenue'].quantile(0.2))&(df_sku_sales['revenue']<= df_sku_sales['revenue'].quantile(0.5))]['original_id'].tolist()

In [94]:
len(list_sku)

467

In [95]:
df_prepilot = df_prepilot.loc[df_prepilot['original_id'].isin(list_sku)].reset_index().drop(columns=['Unnamed: 0', 'index']).copy()

In [96]:
df_pilot = df_pilot.loc[df_pilot['original_id'].isin(list_sku)].reset_index().drop(columns=['Unnamed: 0', 'index']).copy()

In [97]:
df_prepilot.head()

,date,original_id,revenue,total_sku_revenue
0,2023-02-15,14135,1.0,6
1,2023-01-07,14135,0.0,6
2,2023-01-31,14135,1.0,6
3,2023-01-28,14135,1.0,6
4,2023-01-01,14135,0.0,6


In [98]:
df_pilot.head()

,date,original_id,revenue,total_sku_revenue
0,2023-04-16,30618,3.0,77
1,2023-04-15,30618,1.0,77
2,2023-04-17,30618,3.0,77
3,2023-04-19,30618,3.0,77
4,2023-04-14,30618,5.0,77


In [99]:
from sklearn.metrics import mean_squared_error#(y_true, y_pred

In [100]:
from IPython.display import clear_output

In [113]:
#функция для подбора лучших теста и контроля
def best_sample(df, l_sku, n_inter, n_sku):
    iterations = n_inter #число итераций
    var = 100000000000
    
    sample_test_final = []
    sample_control_final = []
    best_iter = 0

    for i in range(iterations):
        print(i, end = ' ')
        clear_output(wait=True)
        df_split = pd.DataFrame({
            'sku':random.sample(list_sku, n_sku),
            'ab_group':np.random.randint(0,2,n_sku)
        })
        sample_try_test = df_split.query('ab_group==1')['sku'].tolist() #рандомный тест
        sample_try_control = df_split.query('ab_group==0')['sku'].tolist()#рандомный контроль
        
        #собираем агрегаты по датам с суммой продаж в штуках
        test_group_check = df.loc[df['original_id'].isin(sample_try_test)] \
            .groupby('date') \
            .agg({'revenue':'sum'}).reset_index()
        
        control_group_check = df.loc[df['original_id'].isin(sample_try_control)] \
            .groupby('date') \
            .agg({'revenue':'sum'}).reset_index()
        
        df_for_diff = test_group_check.merge(control_group_check, how = 'left', on = 'date')#джойн
        print(df_for_diff.revenue_x, ' ', df_for_diff.revenue_y)
        #new_var = mean_squared_error(df_for_diff.revenue_x, df_for_diff.revenue_y)**(1/2)
        new_var = np.std(df_for_diff.revenue_x - df_for_diff.revenue_y)
        if new_var < var:#сохраняем лучший сплит
            var = new_var
            sample_test_final = sample_try_test
            sample_control_final = sample_try_control
            best_iter = i
        
    return sample_try_test, sample_try_control, best_iter

In [114]:
result = best_sample(df_prepilot, list_sku, 5000, 110)

0     22.0
1     25.0
2     14.0
3     17.0
4     34.0
      ... 
95    11.0
96    12.0
97    25.0
98    16.0
99    10.0
Name: revenue_x, Length: 100, dtype: float64   0      6.0
1     11.0
2     16.0
3      4.0
4     13.0
      ... 
95     3.0
96    10.0
97     8.0
98    16.0
99    15.0
Name: revenue_y, Length: 100, dtype: float64


In [106]:
result[2]

2289

In [ ]:
#забираем тест и контроль из функции
final_test = result[0]
final_control = result[1]

test_and_control = final_test + final_control

In [ ]:
#фильтруем наши исходные датафреймы по скю для теста и контроля, присваиваем группы
all_cats_final = pd.DataFrame(all_cats.query('original_id == @test_and_control')).copy()
df_prepilot_final = pd.DataFrame(df_prepilot.query('original_id == @test_and_control')).copy()
df_pilot_final = pd.DataFrame(df_pilot.query('original_id == @test_and_control')).copy()

all_cats_final['groups'] = all_cats_final.original_id.apply(lambda x: 'test' if x in final_test else 'control')
df_prepilot_final['groups'] = df_prepilot_final.original_id.apply(lambda x: 'test' if x in final_test else 'control')
df_pilot_final['groups'] = df_pilot_final.original_id.apply(lambda x: 'test' if x in final_test else 'control')

In [ ]:
#датафрейм для визуализации
vis_groups = all_cats_final.groupby(['groups', 'date']).agg({'revenue':'sum'}).rename(columns = {'revenue':'revenue_sum'}).reset_index()

In [ ]:
sc = StandardScaler()

In [ ]:
#стандартизируем метрики для визуализации
vis_groups_test = vis_groups.loc[vis_groups['groups'] == 'test']
vis_groups_control = vis_groups.loc[vis_groups['groups'] == 'control']

vis_groups_test['rev_standart'] = sc.fit_transform(vis_groups_test[['revenue_sum']])
vis_groups_control['rev_standart'] = sc.fit_transform(vis_groups_control[['revenue_sum']])    

vis_final = pd.concat([vis_groups_test, vis_groups_control])

In [ ]:
f = plt.figure(figsize=(19, 6))
sns.lineplot(data=vis_final, x="date", y="rev_standart", hue = 'groups', ci=None)

In [ ]:
#готовим данные для моделирования по продажам в штуках на наших группах
mtr = 'revenue'
corr_lim = 0, # лимит корреляции

sigma = 1.5
y_train = df_prepilot_final.loc[
    df_prepilot_final['groups'] == 'test'
].pivot_table(index='date', aggfunc={mtr:'sum'})[[mtr]].values
X_train = df_prepilot_final.loc[
    df_prepilot_final['groups'] == 'control'
].pivot_table(index='date', aggfunc={mtr:'sum'}).reset_index()[[mtr]].values
X_train = sm.add_constant(X_train)
    
y_test = df_pilot_final.loc[
    df_pilot_final['groups'] == 'test'
].pivot_table(index='date', aggfunc={mtr:'sum'})[[mtr]].values
X_test = df_pilot_final.loc[
    df_pilot_final['groups'] == 'control'
].pivot_table(index='date', aggfunc={mtr:'sum'}).reset_index()[[mtr]].values
X_test = sm.add_constant(X_test)
    
# обучаем модель
model = sm.OLS(y_train, X_train)
results = model.fit()

In [ ]:
    # оцениваем точность модели
    float_rsquared = round(results.rsquared, 4)
    series_predictions = pd.Series(results.predict(X_test))
    series_facts = pd.Series([i[0] for i in y_test])
    series_prepilot_predictions = pd.Series(results.predict())
    series_prepilot_facts = pd.Series([i[0] for i in y_train])
    rmse = mean_squared_error(series_predictions, series_facts)**(1/2)
    rmse_prec = round(rmse / series_predictions.mean(), 4)
    print(f'R-squared: {float_rsquared}, holdout {sigma} rmse (%): {sigma * rmse_prec}')

    # визуализация точности прогнозирования
    t_prepilot = [i + 1 for i in range(len(series_prepilot_predictions))]
    t_pilot = [i + 1 for i in range(max(t_prepilot), max(t_prepilot) + len(series_predictions))]
    df_pilot_pred = pd.DataFrame({
        'data_type':'forecast',
        'period_type':'pilot',
        'values':series_predictions,
        'ci_low':series_predictions - rmse * sigma,
        'ci_up':series_predictions + rmse * sigma,
        't':t_pilot
    })
    df_pilot_fact = pd.DataFrame({
        'data_type':'fact',
        'period_type':'pilot',
        'values':series_facts,
        'ci_low':series_predictions - rmse * sigma,
        'ci_up':series_predictions + rmse * sigma,
        't':t_pilot
    })
    df_prepilot_pred = pd.DataFrame({
        'data_type':'forecast',
        'period_type':'prepilot',
        'values':series_prepilot_predictions,
        'ci_low':series_prepilot_predictions - rmse * sigma,
        'ci_up':series_prepilot_predictions + rmse * sigma,
        't':t_prepilot
    })
    df_prepilot_fact = pd.DataFrame({
        'data_type':'fact',
        'period_type':'prepilot',
        'values':series_prepilot_facts,
        'ci_low':series_prepilot_predictions - rmse * sigma,
        'ci_up':series_prepilot_predictions + rmse * sigma,
        't':t_prepilot
    })
    df_results = pd.concat([df_prepilot_fact, df_prepilot_pred, df_pilot_fact, df_pilot_pred])
    fig = px.line(
        df_results, 
        x="t", 
        y=['ci_low', "values", 'ci_up'], 
        color='data_type',
        color_discrete_sequence=["blue", "orange"]
    )
    fig.add_vline(
        x=min(t_pilot), 
        line_width=2, 
        line_dash="dash", 
        line_color="black", 
        opacity=0.7,
        annotation_text="A/A period start", 
        annotation_position="top left"
    )
    fig.show()